<a href="https://colab.research.google.com/github/musketeerchess/ChessTables/blob/master/complete_nnue_training_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies

In [1]:
!pip install peewee pytorch-lightning
!pip install python-chess



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 27.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

# connect to google drive

In [1]:
from google.colab import drive
import os
folder_name="/content/mydrive/MyDrive/musketteer_chessresults"
drive.mount("mydrive")
if os.path.exists(folder_name):
  print('folder already exists')
else:
  os.mkdir(folder_name)

ValueError: mount failed

# run helper functions for data preprocessing

In [ ]:
import time
import re
import os
# import chess
import numpy as np
from random import randrange
import json

In [ ]:
def generate_fen(board,catlingrightblack, catlingrightswhite,full_move_counter,halfmove_counter, player):
    fen2=''

    if player =="white":
        fen2=fen2+" b "
    else:
        fen2=fen2+" w "
    if  catlingrightswhite or catlingrightblack:
        if catlingrightswhite:
            fen2=fen2+"KQ"
        if catlingrightblack:
            fen2=fen2+"kq"

    else:
        fen2=fen2+"-"


    fen2 =fen2+" - "+str(halfmove_counter)+" "+str(full_move_counter)

    fen = ''
    empty_count = 0

    for row in board:
        for square in row:
            if square == '.':
                empty_count += 1
            else:
                if empty_count > 0:
                    fen += str(empty_count)
                    empty_count = 0
                fen += square
        if empty_count > 0:
            fen += str(empty_count)
            empty_count = 0
        fen += '/'

    fen = fen[:-1]
    fen = "".join(fen.split(" "))

    return fen.strip()+fen2

def is_valid_move(board, piece, start, end, color):

    def pos_to_coords(pos):
        col, row = pos
        return ord(col) - ord('a'), int(row) - 1
    board.reverse()
    def is_path_clear(start_x, start_y, end_x, end_y):
        if start_x == end_x:
            step = 1 if end_y > start_y else -1
            for y in range(start_y + step, end_y, step):
                if board[y][start_x] != '.':
                    # print("inbetween",board[y][start_x],y,start_x)
                    # print(board)
                    return False
        elif start_y == end_y:
            step = 1 if end_x > start_x else -1
            for x in range(start_x+step , end_x, step):
                if board[start_y][x] != '.':
                    # print("inbetween",board[start_y][x],start_y,x)
                    # print(board)
                    return False
        return True

    start_x, start_y = pos_to_coords(start)
    end_x, end_y = pos_to_coords(end)

    dx = end_x - start_x
    dy = end_y - start_y
    if piece == 'P':  # Pawn
        if color == 'white':
            if start_y == 1 and dy in (1, 2) and dx == 0:
                return True  # First move can be two squares forward for white
            if dy == 1 and dx == 0:
                return True  # Normal move for white
            if dy == 1 and abs(dx) == 1:
                return True  # Capture move for white
        elif color == 'black':
            if start_y == 6 and dy in (-1, -2) and dx == 0:
                return True  # First move can be two squares forward for black
            if dy == -1 and dx == 0:
                return True  # Normal move for black
            if dy == -1 and abs(dx) == 1:
                return True  # Capture move for black
        return False
    elif piece == 'N':  # Knight
        return (abs(dx), abs(dy)) in [(1, 2), (2, 1)]
    elif piece == 'B':  # Bishop
        return abs(dx) == abs(dy)
    elif piece == 'R':  # Rook
        return (dx == 0 or dy == 0) and is_path_clear(start_x, start_y, end_x, end_y)
    elif piece == 'Q':  # Queen
        return (dx == 0 or dy == 0 or abs(dx) == abs(dy))
    elif piece == 'K':  # King
        return max(abs(dx), abs(dy)) == 1
    elif piece == 'A':  # Archbishop (Bishop + Knight)
        return (abs(dx) == abs(dy) and is_path_clear(start_x, start_y, end_x, end_y)) or (abs(dx), abs(dy)) in [(1, 2), (2, 1)]
    elif piece == 'M':  # Chancellor (Rook + Knight)
        return (dx == 0 or dy == 0 and is_path_clear(start_x, start_y, end_x, end_y)) or (abs(dx), abs(dy)) in [(1, 2), (2, 1)]
    elif piece == 'H':  # Hawk
        # print("HHHHHHHHHHHH", abs(dx), abs(dy))
        return (abs(dx), abs(dy)) in [ (2, 0), (0, 2), (3, 0), (0, 3)] or abs(dx) == abs(dy)
    elif piece == 'E':  # Elephant
        return (abs(dx), abs(dy)) == (2, 2) or (abs(dx), abs(dy)) == (2, 0) or (abs(dx), abs(dy)) == (0, 2)
    elif piece == 'U':  # Unicorn (Knight + camel)
        return (abs(dx), abs(dy)) in [(1, 3), (3, 1)] or (abs(dx), abs(dy)) in [(1, 2), (2, 1)]
    else:
        return False


def find_piece_locations(board, piece):
    locations = []
    for row in range(8):
        for col in range(8):
            if board[row][col] == piece:
                locations.append((chr(col + ord('a')), 8 - row))

    return locations

def find_last_p_index(letter,column_data):
        reversed_column = column_data[::-1]
        reverse_index = reversed_column.index(letter)
        original_index = len(column_data) - 1 - reverse_index
        return original_index + 1


def get_min_value(r,k,x):
    if abs(r-x )>abs(k-x):
        return k
    else:
        return r
def update_board(board, movement, player, allow_castling_black,allow_castling_white,full_move_counter,halfmove_counter):
    # Define a dictionary to map algebraic notation to board indices
    halfmove_counter=halfmove_counter+1
    full_move_counter=full_move_counter+1
    algebraic_to_index = {
        'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7,
        '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8
    }
    peices={
        "R":"rook" ,
        "N":"knight" ,
        "B":"Bishop" ,
        "Q":"queen" ,
        "K": "King",
        "H":"hawk",
        "U":"unicorn",

    }

    if "O-O-O"in movement.strip():
        gating=False

        if "/" in movement:
            move=movement.split("/")[0]
            gating=True
            gatingpiece=movement.split("/")[1]


        else:
            move=movement


        if player=="black":
            if allow_castling_black:
                allow_castling_black=False
            start_rook=[1,0]
            start_king=[1,4]
            end_king=[1,2]
            end_rook=[1,3]
            board[start_rook[0]][start_rook[1]]='.'
            board[start_king[0]][start_king[1]]='.'
            board[end_rook[0]][end_rook[1]]='r'
            board[end_king[0]][end_king[1]]='k'

        elif player=="white":
            if allow_castling_white:
                allow_castling_white=False
            start_rook=[8,0]
            start_king=[8,4]
            end_king=[8,2]
            end_rook=[8,3]
            board[start_rook[0]][start_rook[1]]='.'
            board[start_king[0]][start_king[1]]='.'
            board[end_rook[0]][end_rook[1]]='R'
            board[end_king[0]][end_king[1]]='K'

        if gating==True:
            if player=="white":
                gatingpiece1=gatingpiece[0].upper()
                gating_position=gatingpiece[1]

                x_position=board[9].index(gatingpiece1)

                emptttt=get_min_value(start_rook[1],start_king[1],x_position)
                source = (x_position, 9)
                dest = (emptttt,8)

                board[dest[1]] [dest[0]] = board[source[1]][source[0]]


                board[source[1]][source[0]] = '.'
            elif player=="black":
                gatingpiece1=gatingpiece[0].lower()
                gating_position=gatingpiece[1]


                x_position=board[0].index(gatingpiece1)
                emptttt=get_min_value(start_rook[1],start_king[1],x_position)
                source = (x_position,0)

                dest = (emptttt,1)

                board[dest[1]] [dest[0]] = board[source[1]][source[0]]


                board[source[1]][source[0]] = '.'




        return board,allow_castling_black,allow_castling_white,full_move_counter,halfmove_counter


    if "O-O" in movement.strip():

        gating=False

        if "/" in movement:
            move=movement.split("/")[0]
            gating=True
            gatingpiece=movement.split("/")[1]


        else:
            move=movement

        if player=="black":
            if allow_castling_black:
                allow_castling_black=False
            start_rook=[1,7]
            start_king=[1,4]
            end_king=[1,6]
            end_rook=[1,5]
            board[start_rook[0]][start_rook[1]]='.'
            board[start_king[0]][start_king[1]]='.'
            board[end_rook[0]][end_rook[1]]='r'
            board[end_king[0]][end_king[1]]='k'


        elif player=="white":
            if allow_castling_white:
                allow_castling_white=False
            start_rook=[8,7]
            start_king=[8,4]
            end_king=[8,6]
            end_rook=[8,5]
            board[start_rook[0]][start_rook[1]]='.'
            board[start_king[0]][start_king[1]]='.'
            board[end_rook[0]][end_rook[1]]='R'
            board[end_king[0]][end_king[1]]='K'

        if gating==True:
            if player=="white":
                gatingpiece1=gatingpiece[0].upper()
                gating_position=gatingpiece[1]

                x_position=board[9].index(gatingpiece1)
                emptttt=get_min_value(start_rook[1],start_king[1],x_position)
                source = (x_position, 9)
                dest = (emptttt,8)
                # print(source)

                board[dest[1]] [dest[0]] = board[source[1]][source[0]]

                board[source[1]][source[0]] = '.'
            elif player=="black":
                gatingpiece1=gatingpiece[0].lower()
                gating_position=gatingpiece[1]

                x_position=board[0].index(gatingpiece1)
                emptttt=get_min_value(start_rook[1],start_king[1],x_position)
                source = (x_position,0)

                dest = (emptttt,1)


                board[dest[1]] [dest[0]] = board[source[1]][source[0]]

                board[source[1]][source[0]] = '.'

        return board,allow_castling_black,allow_castling_white,full_move_counter,halfmove_counter





    gating=False

    if "/" in movement:
        move=movement.split("/")[0]
        gating=True
        gatingpiece=movement.split("/")[1][0]


    else:
        move=movement

    if "x" in move:
        move=move.replace("x","")
        # print("capture!: ",move)
        halfmove_counter=0

    if "+" in move:
        move=move.replace("+","")
        # print("check: ",move)

    Newpeice=None
    if "=" in move:
        Newpeice=move.split("=")[1]
        move=move.split("=")[0]

        # print("promotioN!: ",move)

    peice_moved="p"
    for x in move:
        if x.isupper():
            peice_moved=x

    move=move.replace(peice_moved,"")




    if player =="white":

        if peice_moved=="p":
            halfmove_counter=0
            if len(move)==3:


                destination=move[-2:]
                start=move[0]
                peice_moved="P"

                locations=find_piece_locations(board[1:-1],peice_moved )
                stringlocs=[]
                for x in locations:
                    stringlocs.append(str(x[0])+str(x[1]))

                for xxxxx in stringlocs:

                    if start in xxxxx and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player) :
                        start=xxxxx

                start=start[0]+str(start[1])






            elif len(move)==4:
                start=move[:2]
                destination=move[2:]




            elif len(move)==2:
                destination=move

                start=move[0]
                peice_moved="P"

                locations=find_piece_locations(board[1:-1],peice_moved )
                stringlocs=[]
                for x in locations:
                    stringlocs.append(str(x[0])+str(x[1]))

                for xxxxx in stringlocs:
                    if start in xxxxx  and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player):
                        start=xxxxx

        else:


            peice_moved=peice_moved.upper()
            if allow_castling_white:
                if peice_moved=="K":
                    allow_castling_white=False
            locations=find_piece_locations(board[1:-1],peice_moved )
            stringlocs=[]
            for x in locations:
                stringlocs.append(str(x[0])+str(x[1]))
            if len(move)==3:
                destination=move[-2:]
                start=move[0]
                for xxxxx in stringlocs:
                    if start in xxxxx and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player) :
                        start=xxxxx
            elif len(move)==4:
                start=move[:2]
                destination=move[2:]
            elif len(move)==2:
                destination=move
                for xxxxx in stringlocs:
                    if  is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player) :
                        start=xxxxx




    elif player =="black":

        halfmove_counter=0
        if peice_moved=="p":

            if len(move)==3:
                destination=move[-2:]
                start=move[0]
                peice_moved="p"

                locations=find_piece_locations(board[1:-1],peice_moved )
                stringlocs=[]
                for x in locations:
                    stringlocs.append(str(x[0])+str(x[1]))

                for xxxxx in stringlocs:

                    if start in xxxxx  and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player):
                        start=xxxxx

                start=start[0]+str(start[1])



            elif len(move)==4:
                start=move[:2]
                destination=move[2:]



            elif len(move)==2:
                destination=move


                start=move[0]
                peice_moved="p"

                locations=find_piece_locations(board[1:-1],peice_moved )
                stringlocs=[]
                for x in locations:
                    stringlocs.append(str(x[0])+str(x[1]))

                for xxxxx in stringlocs:

                    if start in xxxxx  and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player):
                        start=xxxxx

        else:


            peice_moved=peice_moved.lower()
            if allow_castling_black:
                if peice_moved=="k":
                    allow_castling_black=False
            locations=find_piece_locations(board[1:-1],peice_moved )

            stringlocs=[]
            for x in locations:
                stringlocs.append(str(x[0])+str(x[1]))
            if len(move)==3:
                destination=move[-2:]
                start=move[0]
                for xxxxx in stringlocs:
                    if start in xxxxx and is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player) :
                        start=xxxxx
            elif len(move)==4:
                start=move[:2]
                destination=move[2:]
            elif len(move)==2:
                destination=move
                for xxxxx in stringlocs:
                    vvalid=is_valid_move(board[1:-1],peice_moved.upper(), xxxxx, destination,player)

                    if  vvalid :
                        start=xxxxx





    board.reverse()
    if Newpeice:
        source = (algebraic_to_index[start[0]], algebraic_to_index[start[1]])
        dest = (algebraic_to_index[destination[0]], algebraic_to_index[destination[1]])


        board[dest[1]] [dest[0]] = Newpeice


        board[source[1]][source[0]] = '.'

    else:
        source = (algebraic_to_index[start[0]], algebraic_to_index[start[1]])
        dest = (algebraic_to_index[destination[0]], algebraic_to_index[destination[1]])


        board[dest[1]] [dest[0]] = board[source[1]][source[0]]


        board[source[1]][source[0]] = '.'



    if gating==True:
        if player=="black":
            gatingpiece=gatingpiece.lower()

            x_position=board[9].index(gatingpiece)
            source = (x_position, algebraic_to_index[start[1]]+1)
            dest = (algebraic_to_index[start[0]], algebraic_to_index[start[1]])



            board[dest[1]] [dest[0]] = board[source[1]][source[0]]


            board[source[1]][source[0]] = '.'
        elif player=="white":
            gatingpiece=gatingpiece.upper()


            x_position=board[0].index(gatingpiece)

            source = (x_position, algebraic_to_index[start[1]]-1)
            dest = (algebraic_to_index[start[0]], algebraic_to_index[start[1]])


            board[dest[1]] [dest[0]] = board[source[1]][source[0]]

            board[source[1]][source[0]] = '.'










    board.reverse()

    return board ,allow_castling_black,allow_castling_white,full_move_counter,halfmove_counter



def extract_evaluations(moves):
    evaluations = []
    for move in moves:
    # Extract and append white evaluation if it exists
        white_eval_matches = re.findall(r"{([^}]+)}", move)

        if white_eval_matches:
            white_eval_str = white_eval_matches[0]
            # print(white_eval_str)
            try:
              white_eval_value = float(re.findall(r"[-+]?\d*\.\d+|\d+", white_eval_str)[0])
            except:
              pass
            evaluations.append(white_eval_value)
        # Extract and append black evaluation if it exists
        try:
            if len(white_eval_matches) > 1:
                black_eval_str = white_eval_matches[1]
                # print(black_eval_str)
                black_eval_value = -float(re.findall(r"[-+]?\d*\.\d+|\d+", black_eval_str)[0])
                # print(black_eval_value)
                evaluations.append(black_eval_value)
        except:
            pass


    return evaluations





# add location to your pgn file
## (make sure the pgn file is uploaded)


In [ ]:
pathtopgnfile="./correctdataset.pgn"

# run data preprocessing from the pgn file

In [ ]:
with open(pathtopgnfile, 'r') as file:
            pgn_data = file.read()
x=pgn_data.split('[Event "Computer Chess Game"]')[1:]
games=[]


ganesfenres=[]
lenssth=0
for y in x:
    lenssth=lenssth+1
    print("preprocesing data for game: "+str(lenssth))
    gamed=y.split('[SetUp "1"]')
    board=y.split("{--------------")[1].split("--------------}")[0].split("white to play")[0]

    board=board.strip().split("\n")
    newboard=[]
    for x in board:
        newboard.append(x.split(" "))

    pattern = r'(\d+\.\s?[^\d])'
    rere=re.findall(pattern, y.split("--------------}")[1])

    moves=[]
    for  tt  in rere:

        movesssss= tt[-2:]+y.split(tt)[1].split("}",1)[0]+"}"+y.split(tt)[1].split("}",2)[1]+"}"

        moves.append(movesssss.replace("\n","").strip())



    if y.rsplit('}')[-1].strip()== "1-0":
        result=1.0
    elif y.rsplit('}')[-1].strip()== "0-1":
          result=0.0
    elif y.rsplit('}')[-1].strip()== "1/2-1/2":
          result=0.5


    full_move_counter=y.split("FEN")[1].split("]")[0].strip().split(" ")[-1].strip()
    full_move_counter = re.findall(r'\d+', full_move_counter)[0]
    halfmove_counter=y.split("FEN")[1].split("]")[0].strip().split(" ")[-2]



    catlingrightblack=True
    catlingrightswhite=True


    fensssss=[]
    results=result
    # try:
    evaluationssss=extract_evaluations(moves)

    error123=False
    for movess in moves:

        try:
          white=movess.split("{")[0].strip()
          black=movess.split("{")[1].split("}")[1].strip()

          new_board, catlingrightblack, catlingrightswhite,full_move_counter,halfmove_counter = update_board(newboard, white.strip(),"white", catlingrightblack, catlingrightswhite,int(full_move_counter),int(halfmove_counter))

          xxx=(generate_fen(new_board,catlingrightblack, catlingrightswhite,full_move_counter,halfmove_counter, "white"))
          fensssss.append({"fen":xxx,"move":white})

          if black.strip()!="":

                new_board, catlingrightblack, catlingrightswhite,full_move_counter,halfmove_counter = update_board(newboard, black.strip(),"black", catlingrightblack, catlingrightswhite,int(full_move_counter),int(halfmove_counter))

                yyy=(generate_fen(new_board,catlingrightblack, catlingrightswhite,full_move_counter,halfmove_counter, "black"))

                fensssss.append({"fen":yyy,"move":black})

        except:
          error123=True
          break
    if error123==False:
      continue
    ganesfenres.append({"fen":fensssss,"result":results, "evaluations":evaluationssss})


x=json.dumps(ganesfenres,indent=4)
with open("jsondata.json", "w") as f:
    f.write(x)










preprocesing data for game: 1
preprocesing data for game: 2
preprocesing data for game: 3
preprocesing data for game: 4
preprocesing data for game: 5
preprocesing data for game: 6
preprocesing data for game: 7
preprocesing data for game: 8
preprocesing data for game: 9
preprocesing data for game: 10
preprocesing data for game: 11
preprocesing data for game: 12
preprocesing data for game: 13
preprocesing data for game: 14
preprocesing data for game: 15
preprocesing data for game: 16
preprocesing data for game: 17
preprocesing data for game: 18
preprocesing data for game: 19
preprocesing data for game: 20
preprocesing data for game: 21
preprocesing data for game: 22
preprocesing data for game: 23
preprocesing data for game: 24
preprocesing data for game: 25
preprocesing data for game: 26
preprocesing data for game: 27
preprocesing data for game: 28
preprocesing data for game: 29
preprocesing data for game: 30
preprocesing data for game: 31
preprocesing data for game: 32
preprocesing data

# check how much data has been successfully loaded

In [ ]:
len(ganesfenres)

80

# now run Helper functions for AI training

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from collections import OrderedDict
import time
import re
import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset, random_split
import pytorch_lightning as pl
from random import randrange


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
import json

class EvaluationDataset(Dataset):
    def __init__(self, json_file_path):
        """
        Dataset for evaluating chess games.

        Args:
            json_file_path (str): Path to the JSON file containing chess games.
        """
        self.games = self.load_games(json_file_path)
        self.data = self.prepare_data()

    def load_games(self, json_file_path):
        """
        Load games from a JSON file.

        Args:
            json_file_path (str): Path to the JSON file containing chess games.

        Returns:
            list: List of games, where each game is a dictionary.
        """
        with open(json_file_path, 'r') as file:
            games = json.load(file)
        return games

    def prepare_data(self):
        """
        Prepare data by extracting all FEN strings, evaluations, and results.

        Returns:
            list: List of dictionaries containing FEN string, evaluation, and result for each move.
        """
        data = []
        for game in self.games:
            board_states = game['fen']
            evaluations = game['evaluations']
            result = game['result']

            for state, eval_score in zip(board_states, evaluations):
                data.append({
                    'fen': state['fen'],
                    'evaluation': eval_score,
                    'result': result
                })
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        board_state = item['fen']
        evaluation = item['evaluation']

        # Preprocess the board state into binary representation
        binary_board = self.preprocess_board(board_state)

        # Convert evaluation score to a single-element numpy array
        eval_array = np.array([evaluation], dtype=np.float32)

        return {
            'features': torch.tensor(binary_board, dtype=torch.float32),
            'labels': torch.tensor(eval_array, dtype=torch.float32)
        }

    def preprocess_board(self, board_state):
        """
        Preprocess the board state (FEN string) into a fixed-length binary board representation.

        Args:
            board_state (str): FEN string representing the board state.

        Returns:
            np.array: Fixed-length binary board representation.
        """
        # Define a mapping of piece characters to their binary representations
        piece_mapping = {
            '.': [0, 0, 0, 0, 0, 0],  # Empty square
            'p': [0, 0, 0, 0, 0, 1],  # Black pawn
            'r': [0, 0, 0, 0, 1, 0],  # Black rook
            'n': [0, 0, 0, 1, 0, 0],  # Black knight
            'b': [0, 0, 1, 0, 0, 0],  # Black bishop
            'q': [0, 1, 0, 0, 0, 0],  # Black queen
            'k': [1, 0, 0, 0, 0, 0],  # Black king
            'P': [0, 0, 0, 0, 1, 1],  # White pawn
            'R': [0, 0, 0, 1, 0, 1],  # White rook
            'N': [0, 0, 1, 0, 0, 1],  # White knight
            'B': [0, 1, 0, 0, 0, 1],  # White bishop
            'Q': [1, 0, 0, 0, 0, 1],  # White queen
            'K': [0, 0, 1, 1, 0, 0],  # White king
            'H': [1, 0, 0, 1, 1, 0],  # White Hauk
            'U': [0, 1, 1, 0, 0, 0],  # White Unicorn
            'h': [1, 1, 0, 1, 0, 0],  # Black Hauk
            'u': [0, 1, 1, 1, 0, 0]   # Black Unicorn
        }

        # Initialize an empty binary representation of the board
        binary_board = []

        # Split the FEN string into ranks and other parts separated by space
        parts = board_state.split(' ')
        rows = parts[0].split('/')

        # Iterate over each rank in reverse order (starting from the 10th rank)
        for row in reversed(rows):
            # Process each character in the rank
            for char in row:
                # If the character represents a piece, append its binary representation
                if char in piece_mapping:
                    binary_board.extend(piece_mapping[char])
                # If the character represents a numeric count of empty squares, append zeros
                elif char.isdigit():
                    binary_board.extend([0, 0, 0, 0, 0, 0] * int(char))
                elif char != '-':  # Skip dashes
                    raise ValueError(f"Unexpected character in board state: {char}")

        # Pad the binary board to ensure all representations have the same length
        max_length = 6 * 8 * 10  # Maximum length of a Musketeer Chess board
        binary_board.extend([0] * (max_length - len(binary_board)))

        # Convert the binary board to a numpy array for efficiency
        binary_board = np.array(binary_board, dtype=np.float32)

        return binary_board



import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from collections import OrderedDict

class EvaluationModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, batch_size=1024, layer_count=10, json_file_path=None, input_dim=480):
        """
        PyTorch Lightning model for evaluating chess board states.

        Args:
            learning_rate (float): Learning rate for the optimizer.
            batch_size (int): Batch size for training.
            layer_count (int): Number of layers in the neural network.
            json_file_path (str): Path to the JSON file containing chess games.
            input_dim (int): Input dimension size.
        """
        super().__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.json_file_path = json_file_path
        self.layer_count = layer_count

        layers = []
        layers.append((f"linear-0", nn.Linear(input_dim, 808)))
        layers.append((f"relu-0", nn.ReLU()))

        for i in range(1, layer_count - 1):
            layers.append((f"linear-{i}", nn.Linear(808, 808)))
            layers.append((f"relu-{i}", nn.ReLU()))

        layers.append((f"linear-{layer_count - 1}", nn.Linear(808, 1)))
        self.seq = nn.Sequential(OrderedDict(layers))

    def forward(self, x):
        return self.seq(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['features'], batch['labels']
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return [optimizer], [scheduler]

    def train_dataloader(self):
        if not self.json_file_path:
            raise ValueError("JSON file path is not provided.")
        dataset = EvaluationDataset(self.json_file_path)
        return DataLoader(dataset, batch_size=self.batch_size, num_workers=2, pin_memory=True)


# set configuration for the nnue to be trained
## to add a configration simple add the json with the following to the list below

### here you can tweek layer_count, batch_size and max_epochs



example:  {"layer_count": 10, "batch_size": 1024, "json_file_path": "/content/jsondata.json","max_epochs":400}

In [ ]:
configs = [
    {"layer_count": 10, "batch_size": 512, "json_file_path": "/content/jsondata.json","max_epochs":400},
    {"layer_count": 10, "batch_size": 1024, "json_file_path": "/content/jsondata.json","max_epochs":200},
    {"layer_count": 6, "batch_size": 1024, "json_file_path": "/content/jsondata.json","max_epochs":400},
     {"layer_count": 20, "batch_size": 1024, "json_file_path": "/content/jsondata.json","max_epochs":400},
    {"layer_count": 6, "batch_size": 512, "json_file_path": "/content/jsondata.json","max_epochs":400},
    {"layer_count": 20, "batch_size": 512, "json_file_path": "/content/jsondata.json","max_epochs":400}
]

# run trainer it will be stored to you gdrive


In [ ]:

import numpy as np

    # Convert the state_dict to a format suitable for NNUE
    # This is a placeholder conversion function
def convert_to_nnue_format(state_dict):
    nnue_parameters = {}
    for name, param in state_dict.items():
        nnue_parameters[name] = param.cpu().numpy()
    return nnue_parameters


def save_nnue_file(nnue_parameters, file_path):
    with open(file_path, 'wb') as f:
        for name, param in nnue_parameters.items():
            np.save(f, param)
for config in configs:
    version_name = f'{int(time.time())}-batch_size-{config["batch_size"]}-layer_count-{config["layer_count"]}'
    logger = pl.loggers.TensorBoardLogger("lightning_logs", name="chessml", version=version_name)
    trainer = pl.Trainer(
        accelerator='auto',
        strategy='auto',
        devices='auto',
        num_nodes=1,
        precision=16,
        logger=logger,
        callbacks=None,
        fast_dev_run=False,
        max_epochs=config["max_epochs"],
        accumulate_grad_batches=1,
        deterministic=None,
        benchmark=None,
        inference_mode=True,
        use_distributed_sampler=True,
        detect_anomaly=False,
        barebones=False,
        sync_batchnorm=False,
        reload_dataloaders_every_n_epochs=0,
        default_root_dir=None
    )
    model = EvaluationModel(layer_count=config["layer_count"], batch_size=config["batch_size"],
                            learning_rate=1e-3, json_file_path=config["json_file_path"])
    trainer.fit(model)
    model = trainer.model  # Extract the model from the trainer if needed

    state_dict = model.state_dict()

    nnue_parameters = convert_to_nnue_format(state_dict)
    # Save the NNUE parameters to a file
    save_nnue_file(nnue_parameters, '/content/mydrive/MyDrive/musketteer_chessresults/lastFen-model_layers-'+str(config["layer_count"])+"_batchsize-"+str(config["batch_size"])+'.nnue')
    # break  # Remove this if you want to train for all configs




/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:512: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


FileNotFoundError: [Errno 2] No such file or directory: '/content/mydrive/MyDrive/musketteer_chessresults/lastFen-model_layers-10_batchsize-512.nnue'